# This is the first attempt at the Hybrid System

### I'm following along with a lecture on Youtube where someone is detailing how to implement in Python 

Step 1 ) Make a User - Item Iteractions as a sparce matrix with Users as Columns and Items as rows \n
Step 2) Calculate the similarity between Users 
        Works well with alot of data, not with very sparce data
        If Data is sparse performance is no better than random
        Content Based Models to the rescue

    Imports

In [1]:
import numpy as np
import pandas as pd

    Collect Data

In [2]:
reviews = pd.read_csv('data/300kreviews.csv', low_memory=False)

In [3]:
#reviews.drop(columns=['id','verified','style','reviewerName','reviewText','summary'], inplace=True)

In [4]:
#reviews.distinct()

In [5]:
# df2 = reviews['asin'].unique()
# df2
# with open("uniqueasin.txt", "w") as txt_file:
#     for line in df2:
#         txt_file.write("'"+line+"',"+'\n')

    Features approx 610 users, 100,000 Rows and 4 Columns (UserID, itemID, Rating, Timestamp)
        Next part is to collect the Item Names and merge into the  

In [6]:
products = pd.read_csv('data/300k-productinfo.csv')
products.head()

,category,description,title,also_buy,image,also_view,details,main_cat,price,asin
0,[],"[""Loud 'N Clear Personal Sound Amplifier allow...",Loud 'N Clear&trade; Personal Sound Amplifier,[],[],[],"{""ASIN: "":""6546546450""}",All Beauty,NaN,6546546450
1,[],"[""No7 Lift Luminate Triple Action Serum 50ml ...",No7 Lift &amp; Luminate Triple Action Serum 50...,"[""B01E7LCSL6"",""B008X5RVME""]",[],[],"{""Shipping Weight:"":""0.3 ounces ("",""ASIN: "":""7...",All Beauty,$44.99,7178680776
2,[],[],Wella Koleston Perfect Hair Colour 44/44 Mediu...,"[""B0041PBXX8""]","[""https://images-na.ssl-images-amazon.com/imag...",[],"{""\n Item Weight: \n "":""1.76 ounces"",""Sh...",All Beauty,NaN,7367905066
3,[],"[""Lacto Calamine Skin Balance Daily Nourishing...",Lacto Calamine Skin Balance Oil control 120 ml...,[],"[""https://images-na.ssl-images-amazon.com/imag...","[""3254895630"",""B007VL1D9S"",""B00EH9A0RI"",""B0773...","{""Shipping Weight:"":""12 ounces ("",""ASIN: "":""74...",All Beauty,$12.15,7414204790
4,[],"[""Mary Kay Satin Hands Peach Hand Cream Travel...",Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],[],[],"{""Shipping Weight:"":""2.4 ounces"",""ASIN: "":""753...",All Beauty,NaN,7535842801


In [7]:
profiles = pd.read_csv('data/original/profiles.csv')
profiles.head()
# will use this later on though need to add preferences to it

,id,image,user_id,reviewerID
0,NaN,profile_pics/nm0000100_rm1001569280_1955-1-6_2...,1,A3SO2TRTXV835H
1,NaN,profile_pics/nm0000100_rm1002674432_1955-1-6_2...,2,AKPZU0ODODIFB
2,NaN,profile_pics/nm0000100_rm1018346496_1955-1-6_2...,3,A23C0D13EBQ7DN
3,NaN,profile_pics/nm0000100_rm1019451648_1955-1-6_2...,4,ACUAWMMRNI0SH
4,NaN,profile_pics/nm0000100_rm103389184_1955-1-6_20...,5,AMW9MPUISUPBO


## Sanitize the Product Info

In [8]:
products

,category,description,title,also_buy,image,also_view,details,main_cat,price,asin
0,[],"[""Loud 'N Clear Personal Sound Amplifier allow...",Loud 'N Clear&trade; Personal Sound Amplifier,[],[],[],"{""ASIN: "":""6546546450""}",All Beauty,NaN,6546546450
1,[],"[""No7 Lift Luminate Triple Action Serum 50ml ...",No7 Lift &amp; Luminate Triple Action Serum 50...,"[""B01E7LCSL6"",""B008X5RVME""]",[],[],"{""Shipping Weight:"":""0.3 ounces ("",""ASIN: "":""7...",All Beauty,$44.99,7178680776
2,[],[],Wella Koleston Perfect Hair Colour 44/44 Mediu...,"[""B0041PBXX8""]","[""https://images-na.ssl-images-amazon.com/imag...",[],"{""\n Item Weight: \n "":""1.76 ounces"",""Sh...",All Beauty,NaN,7367905066
3,[],"[""Lacto Calamine Skin Balance Daily Nourishing...",Lacto Calamine Skin Balance Oil control 120 ml...,[],"[""https://images-na.ssl-images-amazon.com/imag...","[""3254895630"",""B007VL1D9S"",""B00EH9A0RI"",""B0773...","{""Shipping Weight:"":""12 ounces ("",""ASIN: "":""74...",All Beauty,$12.15,7414204790
4,[],"[""Mary Kay Satin Hands Peach Hand Cream Travel...",Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],[],[],"{""Shipping Weight:"":""2.4 ounces"",""ASIN: "":""753...",All Beauty,NaN,7535842801
...,...,...,...,...,...,...,...,...,...,...
8006,"[""Alexa Skills"",""Games Trivia"",""Knowledge Tr...",[],Florida Facts,[],[],[],{},Alexa Skills,NaN,B01HIYSW0U
8007,"[""Alexa Skills"",""Weather""]",[],TheTemperatureInside,[],[],[],{},Alexa Skills,NaN,B01HJ0BX2C
8008,"[""Alexa Skills"",""Games Trivia"",""Knowledge Tr...",[],Zombie Trivia,[],[],[],{},Alexa Skills,NaN,B01HJ5EDTC
8009,"[""Alexa Skills"",""Education Reference""]",[],Big Data Skills,[],[],[],{},Alexa Skills,NaN,B01HJCS4HC


In [9]:
products['category'] = products['category'].apply(lambda x: x.replace('\"','\''))
products['also_buy'] = products['also_buy'].apply(lambda x: x.replace('\"','\''))
products['image'] = products['image'].apply(lambda x: x.replace('\"','\''))
products['also_view'] = products['also_view'].apply(lambda x: x.replace('\"','\''))


In [10]:
products

,category,description,title,also_buy,image,also_view,details,main_cat,price,asin
0,[],"[""Loud 'N Clear Personal Sound Amplifier allow...",Loud 'N Clear&trade; Personal Sound Amplifier,[],[],[],"{""ASIN: "":""6546546450""}",All Beauty,NaN,6546546450
1,[],"[""No7 Lift Luminate Triple Action Serum 50ml ...",No7 Lift &amp; Luminate Triple Action Serum 50...,"['B01E7LCSL6','B008X5RVME']",[],[],"{""Shipping Weight:"":""0.3 ounces ("",""ASIN: "":""7...",All Beauty,$44.99,7178680776
2,[],[],Wella Koleston Perfect Hair Colour 44/44 Mediu...,['B0041PBXX8'],['https://images-na.ssl-images-amazon.com/imag...,[],"{""\n Item Weight: \n "":""1.76 ounces"",""Sh...",All Beauty,NaN,7367905066
3,[],"[""Lacto Calamine Skin Balance Daily Nourishing...",Lacto Calamine Skin Balance Oil control 120 ml...,[],['https://images-na.ssl-images-amazon.com/imag...,"['3254895630','B007VL1D9S','B00EH9A0RI','B0773...","{""Shipping Weight:"":""12 ounces ("",""ASIN: "":""74...",All Beauty,$12.15,7414204790
4,[],"[""Mary Kay Satin Hands Peach Hand Cream Travel...",Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],[],[],"{""Shipping Weight:"":""2.4 ounces"",""ASIN: "":""753...",All Beauty,NaN,7535842801
...,...,...,...,...,...,...,...,...,...,...
8006,"['Alexa Skills','Games Trivia','Knowledge Tr...",[],Florida Facts,[],[],[],{},Alexa Skills,NaN,B01HIYSW0U
8007,"['Alexa Skills','Weather']",[],TheTemperatureInside,[],[],[],{},Alexa Skills,NaN,B01HJ0BX2C
8008,"['Alexa Skills','Games Trivia','Knowledge Tr...",[],Zombie Trivia,[],[],[],{},Alexa Skills,NaN,B01HJ5EDTC
8009,"['Alexa Skills','Education Reference']",[],Big Data Skills,[],[],[],{},Alexa Skills,NaN,B01HJCS4HC


In [11]:
#sanitize the reviews
reviews

,overall,vote,reviewTime,reviewerID,asin,reviewerName,reviewText,summary
0,1,2,"02 2, 2018",A1U1RE1ZI19E1H,B017O9P72A,Rebekah,this worked great then randomly stopped. pleas...,stopped working
1,5,NaN,"01 15, 2018",A3TXR8GLKS19RE,B017O9P72A,Nello,Great skill,Great
2,4,5,"12 8, 2015",A27ZJ1NCBFP1HZ,B017O9P72A,Greg,"Alexa works great for me so far, but I'm also ...",So Far So Good
3,2,NaN,"12 5, 2015",A3KUPJ396OQF78,B017O9P72A,Larry Russlin,Can only control one of two bulbs from one of ...,Buggy
4,1,5,"12 29, 2017",AA4DHYT5YSSIT,B017O9P72A,angie anj,"The service works with google home, but doesn'...",Does not work
...,...,...,...,...,...,...,...,...
299995,5,NaN,"04 12, 2018",A34F1IZSJIZYM0,B000K10V2Q,ama x 4,This product makes opening locked doors much e...,Helpful for arthritis-impaired hands!!!
299996,2,NaN,"05 14, 2018",AJOO6NSM9IG5P,B000K10V2Q,Espana,This product is not well thought out. Once yo...,I recommend the Key Turner III by North Coast ...
299997,1,NaN,"03 14, 2018",A10XANYF598IRZ,B000K10V2Q,Constance,Waste of money. Cant use with car keys and tha...,Cheap plastic don't bother .
299998,5,NaN,"07 13, 2012",A3PPOVG4CA8XYM,B000JYJIE6,PLS,"This was a great price, and exactly what I was...",Chanel Lipcolors are the best


In [12]:
reviewsandproducts = pd.merge(reviews,products,on='asin')
reviewsandproducts.head()

,overall,vote,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,category,description,title,also_buy,image,also_view,details,main_cat,price
0,1,2,"02 2, 2018",A1U1RE1ZI19E1H,B017O9P72A,Rebekah,this worked great then randomly stopped. pleas...,stopped working,"['Alexa Skills','Smart Home']",[],LIFX,[],[],[],{},Alexa Skills,NaN
1,5,NaN,"01 15, 2018",A3TXR8GLKS19RE,B017O9P72A,Nello,Great skill,Great,"['Alexa Skills','Smart Home']",[],LIFX,[],[],[],{},Alexa Skills,NaN
2,4,5,"12 8, 2015",A27ZJ1NCBFP1HZ,B017O9P72A,Greg,"Alexa works great for me so far, but I'm also ...",So Far So Good,"['Alexa Skills','Smart Home']",[],LIFX,[],[],[],{},Alexa Skills,NaN
3,2,NaN,"12 5, 2015",A3KUPJ396OQF78,B017O9P72A,Larry Russlin,Can only control one of two bulbs from one of ...,Buggy,"['Alexa Skills','Smart Home']",[],LIFX,[],[],[],{},Alexa Skills,NaN
4,1,5,"12 29, 2017",AA4DHYT5YSSIT,B017O9P72A,angie anj,"The service works with google home, but doesn'...",Does not work,"['Alexa Skills','Smart Home']",[],LIFX,[],[],[],{},Alexa Skills,NaN


In [13]:
reviewsandproducts['category'].explode()

0         ['Alexa Skills','Smart Home']
1         ['Alexa Skills','Smart Home']
2         ['Alexa Skills','Smart Home']
3         ['Alexa Skills','Smart Home']
4         ['Alexa Skills','Smart Home']
                      ...              
303553                               []
303554                               []
303555                               []
303556                               []
303557                               []
Name: category, Length: 303558, dtype: object

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

In [15]:
reviews.groupby('asin')['overall'].mean().sort_values(ascending=False).head()

asin
B001ET769O    5.0
B00Z92BLBG    5.0
B00IPYVY4Y    5.0
B006QY8TNY    5.0
B006TFAN70    5.0
Name: overall, dtype: float64

In [16]:
# find the total number of reviews a products has by counting the number or 
reviews.groupby('asin')['overall'].count().sort_values(ascending=False).head()

asin
B000FOI48G    8662
B000GLRREU    8341
1620213982    4792
B001QY8QXM    4534
B01DKQAXC0    4236
Name: overall, dtype: int64

this is done to reccomend the most popular, higest reviewed products to new Users to furthur prevent cold start

In [17]:
ratings = pd.DataFrame(reviews.groupby('asin')['overall'].mean())
ratings.head()

,overall
asin,
0143026860,4.117647
014789302X,4.350000
0692508988,5.000000
0721917704,5.000000
0792272595,5.000000


In [18]:
rating = pd.DataFrame()

so most have very few ratings and some extreme values have 8000

## Recommending Similar Items

In [19]:
#make a function to find the most popular items as a bare minimum 
# The Reccomender formula is (v/(v+m) * R) + (m/(m+v) * C) where : 
#
#
# v = number of reviews an product has gotten - num of ratings
# m = the minimum reviews reqiured to be listed - can find using pandas quartile method - 74.0
#      since only need a few products to be reccomended can use the 90th percentile
# r = the average rating of the product  mean rating 
# c = the mean vote for every product - found using the mean method on the series object for overall (3.940908261807608)

# Need to add the number of ratings using .count() method
ratings['num of ratings'] = pd.DataFrame(reviews.groupby('asin')['overall'].count())
# can add the mean by dividing the total review score from the number of reviews
ratings['mean rating'] = pd.DataFrame(reviews.groupby('asin')['overall'].mean())
ratings.sort_values(by=['num of ratings'], ascending=False)




,overall,num of ratings,mean rating
asin,,,
B000FOI48G,4.392865,8662,4.392865
B000GLRREU,4.405107,8341,4.405107
1620213982,4.798414,4792,4.798414
B001QY8QXM,4.557345,4534,4.557345
B01DKQAXC0,4.208215,4236,4.208215
...,...,...,...
B01DDMC1VC,5.000000,1,5.000000
B001DBL5KY,3.000000,1,3.000000
B01DDJ1HCE,5.000000,1,5.000000


In [20]:
C = ratings['overall'].mean()
print(C)

3.91949504509187


In [21]:
m = ratings['num of ratings'].quantile(0.90)
print(m)

71.0


In [22]:
# Filter out all qualified items into a new DataFrame
q_ratings = ratings.copy().loc[ratings['num of ratings'] >= m]
q_ratings.shape

(803, 3)

In [23]:
# Define the Method used to calculate the rating 
def weighted_rating(x, m=m, C=C):
    v = x['num of ratings']
    R = x['mean rating']
    # Calculation based on the formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [24]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_ratings['score'] = q_ratings.apply(weighted_rating, axis=1)
q_ratings
#Sort Products based on score calculated above
q_ratings = q_ratings.sort_values('score', ascending=False)

#Print the top 15 Products - In the application this will be the products reccomended in the item to item
q_ratings[[ 'num of ratings', 'mean rating', 'score']].head(20)

,num of ratings,mean rating,score
asin,,,
B0009RF9DW,774,4.934109,4.848857
B000FI4S1E,775,4.931613,4.846672
B00020UR4C,459,4.982571,4.840159
B0012Y0ZG2,2224,4.867806,4.838468
B000URXP6E,996,4.852410,4.790332
B0010DMNFQ,304,4.993421,4.790091
1620213982,4792,4.798414,4.785582
B000WZHGVY,312,4.971154,4.776199
B004LUQRJI,243,4.983539,4.742943


so in this case these products would be reccomended to a new User 

## Content Based Filtering Pt 1 - NLP the description

Before the data can be used it needs sanitizing

In [25]:
reviews

,overall,vote,reviewTime,reviewerID,asin,reviewerName,reviewText,summary
0,1,2,"02 2, 2018",A1U1RE1ZI19E1H,B017O9P72A,Rebekah,this worked great then randomly stopped. pleas...,stopped working
1,5,NaN,"01 15, 2018",A3TXR8GLKS19RE,B017O9P72A,Nello,Great skill,Great
2,4,5,"12 8, 2015",A27ZJ1NCBFP1HZ,B017O9P72A,Greg,"Alexa works great for me so far, but I'm also ...",So Far So Good
3,2,NaN,"12 5, 2015",A3KUPJ396OQF78,B017O9P72A,Larry Russlin,Can only control one of two bulbs from one of ...,Buggy
4,1,5,"12 29, 2017",AA4DHYT5YSSIT,B017O9P72A,angie anj,"The service works with google home, but doesn'...",Does not work
...,...,...,...,...,...,...,...,...
299995,5,NaN,"04 12, 2018",A34F1IZSJIZYM0,B000K10V2Q,ama x 4,This product makes opening locked doors much e...,Helpful for arthritis-impaired hands!!!
299996,2,NaN,"05 14, 2018",AJOO6NSM9IG5P,B000K10V2Q,Espana,This product is not well thought out. Once yo...,I recommend the Key Turner III by North Coast ...
299997,1,NaN,"03 14, 2018",A10XANYF598IRZ,B000K10V2Q,Constance,Waste of money. Cant use with car keys and tha...,Cheap plastic don't bother .
299998,5,NaN,"07 13, 2012",A3PPOVG4CA8XYM,B000JYJIE6,PLS,"This was a great price, and exactly what I was...",Chanel Lipcolors are the best


In the Database this will be seperated into Products and reviews 

In [26]:
products

,category,description,title,also_buy,image,also_view,details,main_cat,price,asin
0,[],"[""Loud 'N Clear Personal Sound Amplifier allow...",Loud 'N Clear&trade; Personal Sound Amplifier,[],[],[],"{""ASIN: "":""6546546450""}",All Beauty,NaN,6546546450
1,[],"[""No7 Lift Luminate Triple Action Serum 50ml ...",No7 Lift &amp; Luminate Triple Action Serum 50...,"['B01E7LCSL6','B008X5RVME']",[],[],"{""Shipping Weight:"":""0.3 ounces ("",""ASIN: "":""7...",All Beauty,$44.99,7178680776
2,[],[],Wella Koleston Perfect Hair Colour 44/44 Mediu...,['B0041PBXX8'],['https://images-na.ssl-images-amazon.com/imag...,[],"{""\n Item Weight: \n "":""1.76 ounces"",""Sh...",All Beauty,NaN,7367905066
3,[],"[""Lacto Calamine Skin Balance Daily Nourishing...",Lacto Calamine Skin Balance Oil control 120 ml...,[],['https://images-na.ssl-images-amazon.com/imag...,"['3254895630','B007VL1D9S','B00EH9A0RI','B0773...","{""Shipping Weight:"":""12 ounces ("",""ASIN: "":""74...",All Beauty,$12.15,7414204790
4,[],"[""Mary Kay Satin Hands Peach Hand Cream Travel...",Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],[],[],"{""Shipping Weight:"":""2.4 ounces"",""ASIN: "":""753...",All Beauty,NaN,7535842801
...,...,...,...,...,...,...,...,...,...,...
8006,"['Alexa Skills','Games Trivia','Knowledge Tr...",[],Florida Facts,[],[],[],{},Alexa Skills,NaN,B01HIYSW0U
8007,"['Alexa Skills','Weather']",[],TheTemperatureInside,[],[],[],{},Alexa Skills,NaN,B01HJ0BX2C
8008,"['Alexa Skills','Games Trivia','Knowledge Tr...",[],Zombie Trivia,[],[],[],{},Alexa Skills,NaN,B01HJ5EDTC
8009,"['Alexa Skills','Education Reference']",[],Big Data Skills,[],[],[],{},Alexa Skills,NaN,B01HJCS4HC


The products page has categories that can be used but need sanitizing as features unicode 

In [27]:
for i in products['category']:
     for r in i:
            r.strip('\u0026')

In [28]:
# Replace the price of missing items to $10.00 as it doesnt matter but needs to be working for the web app
products['price'].fillna("$10", inplace=True)
products

,category,description,title,also_buy,image,also_view,details,main_cat,price,asin
0,[],"[""Loud 'N Clear Personal Sound Amplifier allow...",Loud 'N Clear&trade; Personal Sound Amplifier,[],[],[],"{""ASIN: "":""6546546450""}",All Beauty,$10,6546546450
1,[],"[""No7 Lift Luminate Triple Action Serum 50ml ...",No7 Lift &amp; Luminate Triple Action Serum 50...,"['B01E7LCSL6','B008X5RVME']",[],[],"{""Shipping Weight:"":""0.3 ounces ("",""ASIN: "":""7...",All Beauty,$44.99,7178680776
2,[],[],Wella Koleston Perfect Hair Colour 44/44 Mediu...,['B0041PBXX8'],['https://images-na.ssl-images-amazon.com/imag...,[],"{""\n Item Weight: \n "":""1.76 ounces"",""Sh...",All Beauty,$10,7367905066
3,[],"[""Lacto Calamine Skin Balance Daily Nourishing...",Lacto Calamine Skin Balance Oil control 120 ml...,[],['https://images-na.ssl-images-amazon.com/imag...,"['3254895630','B007VL1D9S','B00EH9A0RI','B0773...","{""Shipping Weight:"":""12 ounces ("",""ASIN: "":""74...",All Beauty,$12.15,7414204790
4,[],"[""Mary Kay Satin Hands Peach Hand Cream Travel...",Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],[],[],"{""Shipping Weight:"":""2.4 ounces"",""ASIN: "":""753...",All Beauty,$10,7535842801
...,...,...,...,...,...,...,...,...,...,...
8006,"['Alexa Skills','Games Trivia','Knowledge Tr...",[],Florida Facts,[],[],[],{},Alexa Skills,$10,B01HIYSW0U
8007,"['Alexa Skills','Weather']",[],TheTemperatureInside,[],[],[],{},Alexa Skills,$10,B01HJ0BX2C
8008,"['Alexa Skills','Games Trivia','Knowledge Tr...",[],Zombie Trivia,[],[],[],{},Alexa Skills,$10,B01HJ5EDTC
8009,"['Alexa Skills','Education Reference']",[],Big Data Skills,[],[],[],{},Alexa Skills,$10,B01HJCS4HC


In [29]:
#we dont want any with no image so output the list of affected asins to a text file 
#then drop them from both the products an the review
#print(products['price'] = np.NaN)
# we also dont want any with no price so automatically set the price to be $10 as the actual price isnt important
# also want to sanitize the category and description to remove unicode
products['products'] = products['price'].fillna('$10', inplace=True)
# look at df
products

,category,description,title,also_buy,image,also_view,details,main_cat,price,asin,products
0,[],"[""Loud 'N Clear Personal Sound Amplifier allow...",Loud 'N Clear&trade; Personal Sound Amplifier,[],[],[],"{""ASIN: "":""6546546450""}",All Beauty,$10,6546546450,None
1,[],"[""No7 Lift Luminate Triple Action Serum 50ml ...",No7 Lift &amp; Luminate Triple Action Serum 50...,"['B01E7LCSL6','B008X5RVME']",[],[],"{""Shipping Weight:"":""0.3 ounces ("",""ASIN: "":""7...",All Beauty,$44.99,7178680776,None
2,[],[],Wella Koleston Perfect Hair Colour 44/44 Mediu...,['B0041PBXX8'],['https://images-na.ssl-images-amazon.com/imag...,[],"{""\n Item Weight: \n "":""1.76 ounces"",""Sh...",All Beauty,$10,7367905066,None
3,[],"[""Lacto Calamine Skin Balance Daily Nourishing...",Lacto Calamine Skin Balance Oil control 120 ml...,[],['https://images-na.ssl-images-amazon.com/imag...,"['3254895630','B007VL1D9S','B00EH9A0RI','B0773...","{""Shipping Weight:"":""12 ounces ("",""ASIN: "":""74...",All Beauty,$12.15,7414204790,None
4,[],"[""Mary Kay Satin Hands Peach Hand Cream Travel...",Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],[],[],"{""Shipping Weight:"":""2.4 ounces"",""ASIN: "":""753...",All Beauty,$10,7535842801,None
...,...,...,...,...,...,...,...,...,...,...,...
8006,"['Alexa Skills','Games Trivia','Knowledge Tr...",[],Florida Facts,[],[],[],{},Alexa Skills,$10,B01HIYSW0U,None
8007,"['Alexa Skills','Weather']",[],TheTemperatureInside,[],[],[],{},Alexa Skills,$10,B01HJ0BX2C,None
8008,"['Alexa Skills','Games Trivia','Knowledge Tr...",[],Zombie Trivia,[],[],[],{},Alexa Skills,$10,B01HJ5EDTC,None
8009,"['Alexa Skills','Education Reference']",[],Big Data Skills,[],[],[],{},Alexa Skills,$10,B01HJCS4HC,None


In [30]:
products[['asin', 'title','description']]

,asin,title,description
0,6546546450,Loud 'N Clear&trade; Personal Sound Amplifier,"[""Loud 'N Clear Personal Sound Amplifier allow..."
1,7178680776,No7 Lift &amp; Luminate Triple Action Serum 50...,"[""No7 Lift Luminate Triple Action Serum 50ml ..."
2,7367905066,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[]
3,7414204790,Lacto Calamine Skin Balance Oil control 120 ml...,"[""Lacto Calamine Skin Balance Daily Nourishing..."
4,7535842801,Mary Kay Satin Hands Hand Cream Travel MINI Si...,"[""Mary Kay Satin Hands Peach Hand Cream Travel..."
...,...,...,...
8006,B01HIYSW0U,Florida Facts,[]
8007,B01HJ0BX2C,TheTemperatureInside,[]
8008,B01HJ5EDTC,Zombie Trivia,[]
8009,B01HJCS4HC,Big Data Skills,[]


We will use the categories field, the main_cat field and maybe the description field for content Based Filtering

This can be done using NLP or natural Language Processing to find the Term Frequency-Inverse Document Frequency (TF-IDF) vectors that can be used to find similarities

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

make a copy of the products df to work on 

In [32]:
productstfidf = products.copy()

In [33]:
# remove extraneous english words like 'and' and 'the'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
productstfidf['description'] = productstfidf['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(productstfidf['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(8011, 20474)

In [34]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[5000:5010]

['contiguous',
 'continent',
 'continents',
 'continually',
 'continue',
 'continued',
 'continues',
 'continuing',
 'continuous',
 'continuously']

In [35]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# This compares the matrixes (matricies?) in Item - Item Content Based Filtering based on the english 
# used in the description of the item

In [36]:
cosine_sim.shape

(8011, 8011)

In [37]:
# Now there needs to be a function that takes a product title as an input and outputs the 10
# most similar products - In the web application this will be the products rendered 

#Construct a reverse map of indices and product titles
indices = pd.Series(productstfidf.index, index=productstfidf['title']).drop_duplicates()

In [1]:
indices

NameError: name 'indices' is not defined

In [39]:
indices

title
Loud 'N Clear&trade; Personal Sound Amplifier                                                                                                                     0
No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots                                                                                                         1
Wella Koleston Perfect Hair Colour 44/44 Medium Intense Red Brown 60ml                                                                                            2
Lacto Calamine Skin Balance Oil control 120 ml. (Pack of 2)                                                                                                       3
Mary Kay Satin Hands Hand Cream Travel MINI Size Set of 6                                                                                                         4
                                                                                                                                                               ... 
 Florida F

In [40]:
# Now the reccomendation function can be defined 
# the method needs to : 
#   - Get the index of the product given its title.
#   - Get the list of cosine similarity scores for that particular product with all products.
#     Convert it into a list of tuples where the first element is its position, and the second is the similarity score.
#   - Sort the aforementioned list of tuples based on the similarity scores; that is, the second element
#   - Get the top 10 elements of this list. Ignore the first element as it refers to self 
#     (the product most similar to a particular product is the product itself).
#   - Return the titles or asin corresponding to the indices of the top elements so can be rendered 

# Function that takes in product title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the product that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the products based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar products
    sim_scores = sim_scores[1:11]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar product
    return productstfidf['title'].iloc[product_indices]

In [42]:
get_recommendations("""Loud 'N Clear&trade; Personal Sound Amplifier""")

4594      NYX Cosmetics Big &amp; Loud Lash Primer BLLP01
678                         Willard Water Clear 16 Ounces
1334    Philips Norelco 8040X Moisturizing Shaving System
1987                           ICOM SM-50 DESK MICROPHONE
2297                     XLR 3 Pin Female to Male Adapter
2208    New Hs 300 Wireless Bluetooth Headset Simplifi...
155     Secret Invisible Solid Spring Breeze Twin Pack...
156     Secret Invisible Solid Spring Breeze Twin Pack...
2030    Body Glove Smart Clip Compass and FM Radio wit...
3664    GooDGo Wireless Headphones, Wireless Earphone ...
Name: title, dtype: object

given the input we would expect to see something to do with amplifiers or music which is whats been outputted 

In [44]:
# The second experiment uses a Travel Hand cream to find similar items
get_recommendations("Mary Kay Satin Hands Hand Cream Travel MINI Size Set of 6")

2514                         Mary Kay Compact Cheek Brush
3375    Pecksniffs Gardenia &amp; White Peach Hand &am...
3380    Josie Maran Whipped Argan Oil Intensive Hand C...
2471      Stay On Satin Large Scarf - Style 779-B - Black
2430        Stay On Satin - Drawstring Bonnet: Style 9722
4762                 Stay On Satin Tie Up Wrap Light Pink
7606    Caro White Lightening Beauty Cream with carrot...
5220                       Braun Satin-Hair 7 BR750 brush
1701    Denison Hydrolatum Dry Skin Cream, Soothing Re...
6271    Smashbox X-Rated Mascara - Black - Volumizing ...
Name: title, dtype: object

a brush from the same brand is returned along with some other beauty products

## Content Based Filtering Pt2 - Reccomend Items Based on also viewed/bought products

The amazon dataset comes with a list of also viewed and also bought items that can also be used, though this is a case of returning the asin of those items and rendering them in the webpage

In [45]:
# for example 
from ast import literal_eval
literal_eval(products.at[3, 'also_view'])
#returns a list of also viewed products

['3254895630',
 'B007VL1D9S',
 'B00EH9A0RI',
 'B0773MBG4L',
 'B005631726',
 'B01A0S5326',
 'B076WS9D58',
 'B00F2RUKEW',
 '8974525828',
 'B013BYNHX8',
 'B01MU7URIX',
 'B00KM2P7K4',
 '0335336876',
 'B07CCGW11K',
 'B00L7RLWK2',
 'B01GR475CI',
 'B079STC2NH',
 'B013BYQHM6',
 'B00EH99VY6',
 'B010E1TWGW',
 'B072LVBJHH',
 'B00MV2MO8G',
 'B00OCJ5PUU',
 'B071ZZR11C',
 'B00027D6SE',
 '4453451555',
 'B003TRV2V2',
 'B072J8FNKB',
 'B01MG4PZK0',
 'B01I45XWQQ',
 'B000T5MGF8',
 'B000UDC21C',
 'B019XHPUM8',
 'B01MRKR1G6',
 'B0064FFP96',
 'B00CE731B2',
 'B071SDS4ZM',
 'B07CVQHMG5',
 'B01M7TXCK3',
 'B01BPCAWK4',
 'B0017SWIU4',
 'B07B7KDBZG',
 'B00G0EJYFW',
 'B00NR1YQK4',
 'B07FD34BM8',
 'B00VMYKCL0',
 'B07DD5VQGP',
 'B00NR1YQHM',
 'B017MOR60S',
 'B07FSSYPPJ',
 'B000YJ2SLG',
 'B001V9SXXU',
 'B00X6ZNWG0',
 'B01ES349CY',
 'B0186U9736',
 'B00JXOTBPY',
 'B00CAZAU62']

In [46]:
from ast import literal_eval
literal_eval(products.at[8010, 'also_buy'])
#returns a list of also bought products

['B01HJEGQ6Q',
 'B01L66EYCG',
 'B00H7YZ7HI',
 'B00CHKKWZ4',
 'B00NGYHDF2',
 'B077XWS38X',
 'B01J22AEYS',
 'B077X7NHS5',
 'B011S54ES4',
 'B01DI91VCA',
 'B00CHKKWVI',
 'B0713RP29K',
 'B00P2SZBWM']

The idea will be to have a query to check the size of the also_view and also_buy fields of the database, and to use that if its over a certain number of items, but if it isnt then to use the NLP seen in Part 1 

In [47]:
reviews

,overall,vote,reviewTime,reviewerID,asin,reviewerName,reviewText,summary
0,1,2,"02 2, 2018",A1U1RE1ZI19E1H,B017O9P72A,Rebekah,this worked great then randomly stopped. pleas...,stopped working
1,5,NaN,"01 15, 2018",A3TXR8GLKS19RE,B017O9P72A,Nello,Great skill,Great
2,4,5,"12 8, 2015",A27ZJ1NCBFP1HZ,B017O9P72A,Greg,"Alexa works great for me so far, but I'm also ...",So Far So Good
3,2,NaN,"12 5, 2015",A3KUPJ396OQF78,B017O9P72A,Larry Russlin,Can only control one of two bulbs from one of ...,Buggy
4,1,5,"12 29, 2017",AA4DHYT5YSSIT,B017O9P72A,angie anj,"The service works with google home, but doesn'...",Does not work
...,...,...,...,...,...,...,...,...
299995,5,NaN,"04 12, 2018",A34F1IZSJIZYM0,B000K10V2Q,ama x 4,This product makes opening locked doors much e...,Helpful for arthritis-impaired hands!!!
299996,2,NaN,"05 14, 2018",AJOO6NSM9IG5P,B000K10V2Q,Espana,This product is not well thought out. Once yo...,I recommend the Key Turner III by North Coast ...
299997,1,NaN,"03 14, 2018",A10XANYF598IRZ,B000K10V2Q,Constance,Waste of money. Cant use with car keys and tha...,Cheap plastic don't bother .
299998,5,NaN,"07 13, 2012",A3PPOVG4CA8XYM,B000JYJIE6,PLS,"This was a great price, and exactly what I was...",Chanel Lipcolors are the best


## Collaberative Filtering - Matrix Factorization

Both of the collaberative filtering processes will involve matrix factorization, can do with the reviews dataset

In [48]:
dfcf = pd.DataFrame(data=reviews[['reviewerID','asin','overall']])

In [49]:
dfcf

,reviewerID,asin,overall
0,A1U1RE1ZI19E1H,B017O9P72A,1
1,A3TXR8GLKS19RE,B017O9P72A,5
2,A27ZJ1NCBFP1HZ,B017O9P72A,4
3,A3KUPJ396OQF78,B017O9P72A,2
4,AA4DHYT5YSSIT,B017O9P72A,1
...,...,...,...
299995,A34F1IZSJIZYM0,B000K10V2Q,5
299996,AJOO6NSM9IG5P,B000K10V2Q,2
299997,A10XANYF598IRZ,B000K10V2Q,1
299998,A3PPOVG4CA8XYM,B000JYJIE6,5


In [50]:
# This will be done with the scikit-surprise library
import surprise
from surprise import Dataset
from surprise import Reader

In [51]:
# itemmat = reviews.pivot_table(index='reviewerID',columns='asin',values='overall')
# itemmat
# using this code reesulted in this error :
# MemoryError: Unable to allocate 14.3 GiB for an array with shape (249145, 7685) and data type float64
# so is probably too large, an idea could be to reduce the number of reviews and products used to chunks,
# for example - for a matrix of 300,000 split into 3 matrixes of 100,000 depending on which 
# percent or quartile the product or User falls into

In [52]:
%%time
# Following the above cell I've used only the first 50,000 entries from the reviews dataset to make a matrix
CFDF = reviews.head(100000).copy()
CFDF = CFDF[['reviewerID','asin','overall']]
CFDFmat = CFDF.pivot_table(index='reviewerID',columns='asin',values='overall')
CFDFmat

Wall time: 14.5 s


asin,0143026860,014789302X,0992916305,1620213982,162209798X,4967318355,5874700021,7414204790,7535842801,9790787006,...,B01HGYLRC2,B01HH6R4BW,B01HHBMHG4,B01HHC86Q8,B01HIU7GJW,B01HIUSO7U,B01HIYSW0U,B01HJ0BX2C,B01HJ5EDTC,B01HJCS4HC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A00408825PVJW7GFLEGU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A00891833ZVXH5O654HE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0096681Y127OL1H8W3U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0112699N5AXPSRYATZJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0153524HC25D09HXMND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZOXYRZR81N4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZRXE11AE04G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZS69U1WDPXV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# make a copy of the 
reader = Reader(rating_scale=(0, 5))
# Loads Pandas dataframe
data = Dataset.load_from_df(CFDF, reader)
data

## CF Pt 1 - Item-based Filtering

In [54]:
from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [55]:
# make a training set for the data
# the model require 3 sets - training, validation and test
trainingSet = data.build_full_trainset()

In [56]:
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


#### Testing different algorithms 1 - Algorithms Based on K-Nearest Neighbours (k-NN)

In [57]:
# now the data has been trained we can predict using predict('USER', Item)
# to predict the lift USER would rate Item 
# for the sake of convience I've chosen the first product
prediction = algo.predict('A00408825PVJW7GFLEGU', '7414204790')
prediction

Prediction(uid='A00408825PVJW7GFLEGU', iid='7414204790', r_ui=None, est=4.8, details={'actual_k': 0, 'was_impossible': False})

The prediciton predicts the rating for the item would be 4.4 with the output string :

Prediction(uid='A00408825PVJW7GFLEGU', iid='7414204790', r_ui=None, est=4.4, details={'actual_k': 0, 'was_impossible': False})

#### Testing different algorithms 2 - Algorithms Based SVD Matrix Factorization

In [58]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(CFDF, reader)

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

1.3218704409176412
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


n_epochs : is the number of iterations of SGD, which is basically an iterative method used in statistics to minimize a function.


lr_all : is the learning rate for all parameters, which is a parameter that decides how much the parameters are adjusted in each iteration.


reg_all : is the regularization term for all parameters, which is a penalty term added to prevent overfitting.

### Try with the full dataset

In [59]:
reviews

,overall,vote,reviewTime,reviewerID,asin,reviewerName,reviewText,summary
0,1,2,"02 2, 2018",A1U1RE1ZI19E1H,B017O9P72A,Rebekah,this worked great then randomly stopped. pleas...,stopped working
1,5,NaN,"01 15, 2018",A3TXR8GLKS19RE,B017O9P72A,Nello,Great skill,Great
2,4,5,"12 8, 2015",A27ZJ1NCBFP1HZ,B017O9P72A,Greg,"Alexa works great for me so far, but I'm also ...",So Far So Good
3,2,NaN,"12 5, 2015",A3KUPJ396OQF78,B017O9P72A,Larry Russlin,Can only control one of two bulbs from one of ...,Buggy
4,1,5,"12 29, 2017",AA4DHYT5YSSIT,B017O9P72A,angie anj,"The service works with google home, but doesn'...",Does not work
...,...,...,...,...,...,...,...,...
299995,5,NaN,"04 12, 2018",A34F1IZSJIZYM0,B000K10V2Q,ama x 4,This product makes opening locked doors much e...,Helpful for arthritis-impaired hands!!!
299996,2,NaN,"05 14, 2018",AJOO6NSM9IG5P,B000K10V2Q,Espana,This product is not well thought out. Once yo...,I recommend the Key Turner III by North Coast ...
299997,1,NaN,"03 14, 2018",A10XANYF598IRZ,B000K10V2Q,Constance,Waste of money. Cant use with car keys and tha...,Cheap plastic don't bother .
299998,5,NaN,"07 13, 2012",A3PPOVG4CA8XYM,B000JYJIE6,PLS,"This was a great price, and exactly what I was...",Chanel Lipcolors are the best


In [60]:
fullcfdf = reviews.copy()
fullcfdf =  fullcfdf[['reviewerID','asin','overall']]
fullcfdf


,reviewerID,asin,overall
0,A1U1RE1ZI19E1H,B017O9P72A,1
1,A3TXR8GLKS19RE,B017O9P72A,5
2,A27ZJ1NCBFP1HZ,B017O9P72A,4
3,A3KUPJ396OQF78,B017O9P72A,2
4,AA4DHYT5YSSIT,B017O9P72A,1
...,...,...,...
299995,A34F1IZSJIZYM0,B000K10V2Q,5
299996,AJOO6NSM9IG5P,B000K10V2Q,2
299997,A10XANYF598IRZ,B000K10V2Q,1
299998,A3PPOVG4CA8XYM,B000JYJIE6,5


In [61]:
# make a copy of the 
reader = Reader(rating_scale=(0, 5))
# Loads Pandas dataframe
data = Dataset.load_from_df(fullcfdf, reader)
# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [62]:
# Trying the same test as before 
prediction = algo.predict('A00408825PVJW7GFLEGU', '7414204790')
prediction

Prediction(uid='A00408825PVJW7GFLEGU', iid='7414204790', r_ui=None, est=4.4, details={'actual_k': 0, 'was_impossible': False})

In [63]:
same result

SyntaxError: invalid syntax (<ipython-input-63-991650123fef>, line 1)

## CF Pt 2 - User-based Filtering

In [64]:
# # To use item-based cosine similarity
# sim_options = {
#     "name": "cosine",
#     "user_based": True,  # Compute  similarities between items
# }
# algo = KNNWithMeans(sim_options=sim_options)
# trainingSet = data.build_full_trainset()
# algo.fit(trainingSet)


In [65]:
# attempting to use User Collaberative Filtering gave the following result - 
# MemoryError: Unable to allocate 462. GiB for an array with shape (38496, 38496) and data type float64
# Will use the User Preferences to reccomend them the highest rated entry from that category instead

## Evaluating Model Accuracy and Timing the different Algorithms 

In [66]:
from surprise.model_selection.validation import cross_validate

In [67]:
# make a copy of the 
reader = Reader(rating_scale=(0, 5))
# Loads Pandas dataframe
data = Dataset.load_from_df(fullcfdf, reader)
# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [68]:
fullcfdf.shape

(300000, 3)

In [69]:
# Evaluate use with KNN 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3200  1.3203  1.3178  1.3147  1.3272  1.3200  0.0041  
MAE (testset)     0.9923  0.9931  0.9918  0.9897  0.9990  0.9932  0.0031  
Fit time          1.38    1.70    1.43    1.40    1.48    1.48    0.11    
Test time         0.54    0.82    0.71    0.78    0.78    0.73    0.10    


{'test_rmse': array([1.32001003, 1.32028315, 1.31778929, 1.31465153, 1.32719647]),
 'test_mae': array([0.99227235, 0.99308888, 0.99180606, 0.98965744, 0.99903732]),
 'fit_time': (1.377993106842041,
  1.6962471008300781,
  1.4311943054199219,
  1.4003913402557373,
  1.4770362377166748),
 'test_time': (0.5426206588745117,
  0.8194749355316162,
  0.7121624946594238,
  0.7819907665252686,
  0.7776954174041748)}

In [70]:
# Retrain the model using the entire dataset
trainset = data.build_full_trainset()
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [72]:
# make a copy of the 
reader = Reader(rating_scale=(0, 5))
# Loads Pandas dataframe
data = Dataset.load_from_df(fullcfdf, reader)
# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = SVD(sim_options=sim_options)
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)

TypeError: __init__() got an unexpected keyword argument 'sim_options'

In [74]:
# Change this one
trainingSet = data.build_full_trainset()
algo = SVD()
algo.fit(trainingSet)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2347  1.2355  1.2300  1.2340  1.2389  1.2346  0.0028  
MAE (testset)     0.9515  0.9500  0.9479  0.9514  0.9529  0.9507  0.0017  
Fit time          15.07   14.81   15.14   15.07   15.31   15.08   0.16    
Test time         0.43    0.81    0.37    0.65    0.59    0.57    0.16    


{'test_rmse': array([1.23473473, 1.23548498, 1.23004909, 1.23401555, 1.23889447]),
 'test_mae': array([0.95145055, 0.94998163, 0.94791663, 0.951417  , 0.95290797]),
 'fit_time': (15.068042993545532,
  14.813995838165283,
  15.135570049285889,
  15.07191777229309,
  15.314335823059082),
 'test_time': (0.42731642723083496,
  0.8146450519561768,
  0.36801624298095703,
  0.6471226215362549,
  0.5887339115142822)}

In [81]:
# Evaluating with Gridsearch 
reader = Reader(rating_scale=(0, 5))
# Loads Pandas dataframe
data = Dataset.load_from_df(fullcfdf, reader)
param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
algo = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)
# Change this one
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

AttributeError: 'Trainset' object has no attribute 'raw_ratings'

In [82]:
data = Dataset.load_from_df(fullcfdf, reader)
trainset = data.build_full_trainset()

param_grid = {'n_epochs': [15, 20, 25, 30, 35, 40, 45, 50],
'n_factors': [100, 200, 300],
'lr_all': [0.002, 0.003, 0.004, 0.005],
'reg_all': [0.02,0.01,0.05,0.1, 0.4, 0.6]}
svd = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs = -1)

svd.fit(data)

print("RMSE score", svd.best_score['rmse'])

print("Best Parameters", svd_mf.best_params['rmse'])

KeyboardInterrupt: 

In [76]:
cross_validate(gs, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

AttributeError: 'Trainset' object has no attribute 'raw_ratings'

The fastest was KNN

## Making Predictions and Reccomendations

In [ ]:
# Making New Column for Reccomendations for a User
review2 = reviews.copy()
review2

In [ ]:
# Make a reccomendation for User A1U1RE1ZI19E1H
review2['Estimate_Score'] = review2['asin'].apply(lambda x: algo.predict('A1U1RE1ZI19E1H', x).est)
review2

In [ ]:
# Sort the results and reccomend Items
review2['Estimate_Score'] = review2['asin'].apply(lambda x: algo.predict('A1U1RE1ZI19E1H', x).est)
review2 = review2.sort_values(by=['Estimate_Score'], ascending=False)
x = review2.groupby('Estimate_Score').asin.nunique()
x.plot()

In [ ]:
# Problem that the same product is being reccomended over and over
# reccomends  the first 20 unique asin values 
review2.asin.unique()[0:20]


In [ ]:
# Try with another reviewer 
review2['Estimate_Score'] =  review2['asin'].apply(lambda x: algo.predict('A5PFMT0YEZV32', x).est)
review2.sort_values(by=['Estimate_Score'], ascending=False)

In [ ]:
# Try with another reviewer 
review2['Estimate_Score'] =  review2['asin'].apply(lambda x: algo.predict('AZ6TDFT66CSVW', x).est)
review2.sort_values(by=['Estimate_Score'], ascending=False)
x = review2.groupby('Estimate_Score').asin.nunique()
x.plot()

In [ ]:
review2['Estimate_Score'] =  review2['asin'].apply(lambda x: algo.predict('A38UO7P7BVYUH4', x).est)
review2.sort_values(by=['Estimate_Score'], ascending=False)

In [ ]:
x = review2.groupby('Estimate_Score').asin.nunique()
x.plot()

In [ ]:
# From the graph can see that alot of values have 5 but there is a wide distribution of estimated scores.

In [ ]:
# make a amethod that calls the algo, makes predictions for items
def reccomend(asin):
    

In [ ]:
import joblib

In [ ]:
joblib.dump(algo, 'DjangoModel.pk2')

In [ ]:
modelReload = joblib.load('DjangoModel.pk1')

In [ ]:
modelReload.predict('A38UO7P7BVYUH4', 'B01H7YN8UK')
# this user ID and Product ID combo gave a 5 last time

In [ ]:
modelReload.predict('A1EQ95NWWLH0BW', 'B01EMTFKEU')
# this user ID and Product ID combo gave a 1 last time


In [ ]:
# NICE IT WORKS

In [ ]:
# Now output the data that will be used in the Django Application 
# csvs for database upload then also json for loading in via a python script
reviews.to_csv('data/reviews_model1.csv', index=False)
reviews.to_json('data/reviews_model1.json')
products.to_csv('data/products_model1.csv', index=False)
products.to_json('data/products_model1.json')
profiles.to_csv('data/profiles_model1.csv', index=False)
profiles.to_json('data/profiles_model1.json')

In [ ]:
import json
#reviews.head(10).to_json('data/output/reviews_model2.json')
reviews.to_json(path_or_buf='data/output/reviews_model1.json', orient="records")
products.to_json(path_or_buf='data/output/products_model1.json', orient="records")
profiles.to_json(path_or_buf='data/output/profiles_model1', orient="records")


## TO DO
1) Train on the whole dataset - The WHOLE BIG MASSIVE ONE (aprox 10x this one) 

2) Ouptut the model again

3) Research how to use with django

4) Implement Model 1 on Django 

5) Redo with the whole Dataset - FML 

In [ ]:
products

In [ ]:
products

In [ ]:
products

# JUNK - THE BELOW IS LEFTOVER CODE FROM TESTING AND CONFIGURATION

most rated Item 